In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.sql.functions import *
from pyspark.sql.types import *




In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from time import time

In [4]:
#lines = spark.read.csv("gs://dataset-rs/dataset-last.fm/dataset_lastfm.csv", header="true",inferSchema="true").rdd
lines = spark.read.csv("/home/aleja/Documentos/datasets/meu-lastfm/dataset_lastfm_liked_idsCovert.csv", header="true",inferSchema="true").rdd

lines.take(2)

[Row(_c0=0, user_name='fernandomaiscal', artist_name='Kenny Rogers', track_name='All I Ever Need Is You (feat. Dottie West)', liked='1', new_userId='0', new_songId=47914, new_artistId=17547),
 Row(_c0=1, user_name='fernandomaiscal', artist_name='Geraldo Azevedo', track_name='Dia Branco', liked='1', new_userId='0', new_songId=128432, new_artistId=1637)]

In [5]:
#Create dataframe
ratings = spark.createDataFrame(lines) 
#ratings = ratings.withColumn("topTracks_playcount", col("topTracks_playcount").cast('int'))



In [6]:
#50% Sample of data 
df_ratings = ratings.sample(False,fraction=0.9, seed=1)
#Drop NaN data 
df_ratings = df_ratings.na.drop()
df_ratings.select('new_userId').count()

1065102

In [7]:
df_ratings.show(5)

+---+---------------+---------------+--------------------+-----+----------+----------+------------+
|_c0|      user_name|    artist_name|          track_name|liked|new_userId|new_songId|new_artistId|
+---+---------------+---------------+--------------------+-----+----------+----------+------------+
|  0|fernandomaiscal|   Kenny Rogers|All I Ever Need I...|    1|         0|     47914|       17547|
|  1|fernandomaiscal|Geraldo Azevedo|          Dia Branco|    1|         0|    128432|        1637|
|  2|fernandomaiscal|    John Lennon|    Gimme Some Truth|    1|         0|      9338|        6482|
|  3|fernandomaiscal|    David Bowie|  Station to Station|    1|         0|    170601|        7084|
|  5|fernandomaiscal|   Scott Walker|          The Bridge|    1|         0|     23534|       13436|
+---+---------------+---------------+--------------------+-----+----------+----------+------------+
only showing top 5 rows



In [8]:
df_ratings.printSchema()

root
 |-- _c0: long (nullable = true)
 |-- user_name: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- track_name: string (nullable = true)
 |-- liked: string (nullable = true)
 |-- new_userId: string (nullable = true)
 |-- new_songId: long (nullable = true)
 |-- new_artistId: long (nullable = true)



In [9]:
df_ratings = df_ratings.withColumn("new_userId", col("new_userId").cast("int"))
df_ratings = df_ratings.withColumn("new_songId", col("new_songId").cast("int"))
df_ratings = df_ratings.withColumn("new_artistId", col("new_artistId").cast("int"))
df_ratings = df_ratings.withColumn("liked", col("liked").cast("int"))





In [10]:
df_ratings.printSchema()

root
 |-- _c0: long (nullable = true)
 |-- user_name: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- track_name: string (nullable = true)
 |-- liked: integer (nullable = true)
 |-- new_userId: integer (nullable = true)
 |-- new_songId: integer (nullable = true)
 |-- new_artistId: integer (nullable = true)



In [11]:
## How many distinct users in data ?

uniqueUsers = df_ratings.select('new_userId').distinct().count()
print("Total n. of users: ", uniqueUsers)

## How many distinct artists in data ?

uniqueArtists  = df_ratings.select("new_artistId").distinct().count()
print("Total n. of artists: ", uniqueArtists)

## How many distinct music in data ?

uniqueSongs  = df_ratings.select("new_songId").distinct().count()
print("Total n. of songs: ", uniqueSongs)




Total n. of users:  6227
Total n. of artists:  23267
Total n. of songs:  198051


### TOTAL PLAY COUNT OF EACH TOP_TRACK

#song_play1 = df_ratings.selectExpr("track_name","artist_name").groupBy(col('tracks_name'),col('artist_name')).sum("Total_count")
#song_play1.orderBy("sum(Total_count)",ascending=False).show(10)

#song_play1

#How many times each user has played a song?
# Compute user activity
# We are interested in how many playcounts each user has scored.

#userActivity =  df_ratings.groupBy("new_userId").sum("topTracks_playcount").collect()
#userActivity[0:5]

## Plot CDF (or ECDF) of number of play counts per User ID.
pdf = pd.DataFrame(data=userActivity)
Y=np.sort( pdf[1] )
yvals=np.arange(len(Y))/float(len(Y))

print(np.arange(len(Y)))

plt.plot( Y, yvals )
plt.xlabel('Play Counts')
plt.xlim(0,30000)   # We shrink the window 
plt.ylabel('ECDF')
plt.grid(True,which="both",ls="-")
plt.title('ECDF of number of play counts per User ID')
plt.show()


# Additional relevant code
percentiles = [0.03, 0.10, 0.25, 0.50, 0.75, 0.90]
for p in percentiles:
    i = np.where(yvals<=p)[0][-1]    
    print(str(p*100),"percent of the users has less than: ",Y[i]," play counts")

# Compute artist popularity
# We are interested in how many playcounts per artist
# ATTENTION! Grouping by artistID may be problematic, as stated above.

artistPopularity = df_ratings.groupby("new_artistId").sum("topTracks_playcount").collect()

pdf = pd.DataFrame(data=artistPopularity)
Y=np.sort( pdf[1] )
yvals=np.arange(len(Y))/float(len(Y))

print(np.arange(len(Y)))

plt.plot( Y, yvals )
plt.xlabel('Play Counts')
plt.xlim(0,2000)
plt.ylabel('ECDF')
plt.grid(True,which="both",ls="-")
plt.title('ECDF of number of play counts per Artist ID')
plt.show()

# Compute music popularity
# We are interested in how many playcounts per music
# ATTENTION! Grouping by artistID may be problematic, as stated above.

musicPopularity = df_ratings.groupby("new_songId").sum("topTracks_playcount").collect()

pdf = pd.DataFrame(data=musicPopularity)
Y=np.sort( pdf[1] )
yvals=np.arange(len(Y))/float(len(Y))

print(np.arange(len(Y)))

plt.plot( Y, yvals )
plt.xlabel('Play Counts')
plt.xlim(0,1000)
plt.ylabel('ECDF')
plt.grid(True,which="both",ls="-")
plt.title('ECDF of number of play counts per music ID')
plt.show()

# Additional relevant code
percentiles = [0.03, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95]
for p in percentiles:
    i = np.where(yvals<=p)[0][-1]    
    print(str(p*100),"percent of the music has less than: ",Y[i]," playcount ")

### Total count of  a song listened by users

song_play =df_ratings.groupBy("topTracks_name", "topTracks_artist_name").count()
song_play.orderBy("count", ascending=False).show()

df_ratings.selectExpr("new_songId", "new_artistId",'topTracks_playcount').distinct().describe().show()

grouped_sum0 = song_play.selectExpr('count')
grouped_sum0 = grouped_sum0.groupBy().sum().collect()[0][0]
grouped_sum0


y = song_play.withColumn("porcentage", (song_play['count']/grouped_sum0)*100)
#song_play.selectExpr((song_play['count'] / grouped_sum0)*100)
y.orderBy("count", ascending=False).show()


### create music data

In [12]:
music_data =  df_ratings.selectExpr("new_songId","track_name","new_artistId","artist_name")
music_data

DataFrame[new_songId: int, track_name: string, new_artistId: int, artist_name: string]

### Split the dataset Training and test data

In [13]:
#sub_rating_data = df_ratings_with_int_ids.select("new_userId","new_songId","topTracks_playcount")
sub_rating_data = df_ratings.select("new_userId","new_songId","liked")

sub_rating_data.orderBy(col('new_userId'),col('new_songId')).show(10)


+----------+----------+-----+
|new_userId|new_songId|liked|
+----------+----------+-----+
|      null|         1| null|
|      null|         1| null|
|         0|      1055|    1|
|         0|      1078|    1|
|         0|      1443|    1|
|         0|      1819|    1|
|         0|      2571|    1|
|         0|      4642|    1|
|         0|      4845|    1|
|         0|      5121|    1|
+----------+----------+-----+
only showing top 10 rows



In [14]:
sub_rating_data = sub_rating_data.na.drop()


sub_rating_data.orderBy(col('new_userId'),col('new_songId')).show(10)

+----------+----------+-----+
|new_userId|new_songId|liked|
+----------+----------+-----+
|         0|      1055|    1|
|         0|      1078|    1|
|         0|      1443|    1|
|         0|      1819|    1|
|         0|      2571|    1|
|         0|      4642|    1|
|         0|      4845|    1|
|         0|      5121|    1|
|         0|      5332|    1|
|         0|      5438|    1|
+----------+----------+-----+
only showing top 10 rows



In [15]:
(training, test) = sub_rating_data.randomSplit([0.8, 0.2])
training.printSchema()

root
 |-- new_userId: integer (nullable = true)
 |-- new_songId: integer (nullable = true)
 |-- liked: integer (nullable = true)



In [16]:
#training.orderBy(col('new_userId'),col('new_songId')).show(10)

In [17]:
#test.orderBy(col('new_userId'),col('new_songId')).show(10)

### Treinando ALS

In [18]:
als = ALS(rank=10, maxIter=10, regParam=0.01, alpha=0.5, implicitPrefs=False, userCol="new_userId", itemCol="new_songId", ratingCol="liked", coldStartStrategy="drop")
model = als.fit(training)


In [19]:
##Predictions-  test model
predictions = model.transform(test)

#predictions.show(10)


In [20]:
predictions.printSchema()

root
 |-- new_userId: integer (nullable = true)
 |-- new_songId: integer (nullable = true)
 |-- liked: integer (nullable = true)
 |-- prediction: float (nullable = true)



In [21]:

predictions.show()


+----------+----------+-----+----------+
|new_userId|new_songId|liked|prediction|
+----------+----------+-----+----------+
|      1708|       148|    1| 0.9932618|
|      1978|       148|    1| 0.9956291|
|      4738|       148|    1| 0.9882174|
|      4212|       148|    1|0.99797666|
|      1846|       148|    1|  0.976412|
|      2641|       148|    1| 0.9949031|
|      2624|       148|    1| 1.0010266|
|      6181|       833|    1| 0.9924753|
|      1658|       833|    1|0.96309763|
|      5729|      1342|    1|0.98549324|
|       986|      1959|    1| 0.9639661|
|      1894|      2122|    1|0.97267795|
|      3882|      3749|    1| 0.9971295|
|      4829|      4900|    1| 1.0190439|
|       822|      4900|    1|0.99367636|
|      1613|      4900|    1| 1.0287379|
|      1219|      4900|    1|0.99045247|
|       543|      4900|    1|0.98880726|
|      4071|      5518|    1| 0.9558742|
|      1108|      6336|    1| 0.9784731|
+----------+----------+-----+----------+
only showing top

### Accuracia do modelo

In [22]:
evaluator = RegressionEvaluator(labelCol="liked", predictionCol="prediction")
rmse = evaluator.evaluate(predictions.na.drop(), {evaluator.metricName :"rmse"})
mae =  evaluator.evaluate(predictions.na.drop(), {evaluator.metricName :"mae"})
print("Root-mean-square error = " + str(rmse))
print("mae = " + str(mae))



Root-mean-square error = 0.08053935843481734
mae = 0.027569026808437702


In [23]:
x= model.recommendForAllUsers(10)


In [24]:
x.show(1, truncate=False)

+----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|new_userId|recommendations                                                                                                                                                                                  |
+----------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1580      |[[68207,1.1615322], [75263,1.1457906], [126087,1.136142], [101064,1.1299186], [148559,1.1091481], [49892,1.1052668], [135392,1.0986837], [173982,1.0947129], [15461,1.0943902], [8512,1.0808991]]|
+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------

## Función para recomendación

In [25]:
from pyspark.sql.functions import lit



def recommendMusic(model, user, nbRecommendations):
     # Create a Spark DataFrame with the specified user and all the movies listed in the ratings DataFrame
    dataSet = df_ratings.select("new_songId").distinct().withColumn("new_userId", lit(user))

    # Create a Spark DataFrame with the movies that have already been rated by this user
    musicAlreadyRated = df_ratings.filter(df_ratings.new_userId == user).select("new_songId", "new_userId")

    # Apply the recommender system to the data set without the already rated movies to predict ratings
    predictions = model.transform(dataSet.subtract(musicAlreadyRated)).dropna().select("new_songId", "prediction").orderBy("prediction", ascending=False).limit(nbRecommendations)
    
    # Join with the ratings DataFrame to get the music titles and genres
    recommendations = predictions.join(music_data, predictions.new_songId == music_data.new_songId).select(predictions.new_songId, music_data.track_name, music_data.artist_name, predictions.prediction).orderBy("prediction", ascending=False).distinct()
    recommendations.show(truncate=False)

In [26]:
print ("Recommendations for user 1:")
recommendMusic(model,1,10)


Recommendations for user 367:
+----------+-------------------------------------------------+------------------+----------+
|new_songId|track_name                                       |artist_name       |prediction|
+----------+-------------------------------------------------+------------------+----------+
|68207     |Fishing Bird (Empty Gutted in the Evening Breeze)|Grouper           |1.1550529 |
|126087    |Beyond the Veil                                  |Fall of Efrafa    |1.1450307 |
|75263     |Sarajevo                                         |Max Richter       |1.1344285 |
|101064    |Odessa                                           |Animals as Leaders|1.1140392 |
|49892     |Constants Are Changing                           |Boards of Canada  |1.0984461 |
|148559    |Kő koppan                                        |Thy Catafalque    |1.0947309 |
|135392    |Abysmo                                           |Moonspell         |1.0907323 |
|166403    |Nascence                    

In [27]:
print ("Recommendations for user 48:")
recommendMusic(model,8,10)


Recommendations for user 48:
+----------+-------------------------------------------------+------------------+----------+
|new_songId|track_name                                       |artist_name       |prediction|
+----------+-------------------------------------------------+------------------+----------+
|166403    |Nascence                                         |Austin Wintory    |1.1570499 |
|68207     |Fishing Bird (Empty Gutted in the Evening Breeze)|Grouper           |1.128329  |
|49892     |Constants Are Changing                           |Boards of Canada  |1.1059399 |
|101064    |Odessa                                           |Animals as Leaders|1.0907942 |
|126087    |Beyond the Veil                                  |Fall of Efrafa    |1.0852691 |
|173982    |Autumn Leaves                                    |Bill Evans        |1.0836991 |
|44150     |Give In                                          |The Bravery       |1.0811712 |
|52765     |Electricity                  

### para mejorar la Acuracia del modelo

evaluations = []
#rank [10:50], lambda [1.0, 0.0001], alpha [1.0, 40.0]

for rank in [10,50]:
    for lambda_ in [1.0,0.0001]:
        for alpha in [1.0,40.0]:
            print("Train model with rank=%d lambda_=%f alpha=%f" % (rank, lambda_, alpha))
            model = ALS(rank=rank, maxIter=15, regParam=lambda_, alpha=alpha, implicitPrefs=True,  
                        userCol="new_userId", itemCol="new_songId", ratingCol="topTracks_playcount", coldStartStrategy="drop").fit(training)
            #model = als.fit(training)
            predictions = model.transform(test)
            evaluator = RegressionEvaluator(labelCol="topTracks_playcount", predictionCol="prediction")
            rmse = evaluator.evaluate(predictions.na.drop(), {evaluator.metricName :"rmse"})
            
            evaluations.append(((rank, lambda_, alpha), rmse))
            
            #unpersist(model)

evaluations.sort(key=lambda r:r[1], reverse = True)
                 
evalDataFrame = pd.DataFrame(data=evaluations)
print(evalDataFrame)


            
#training.unpersist()
#test.unpersist()           

#performance 

def kfoldALS(data, k=3, userCol="new_userId", itemCol="new_songId", ratingCol="topTracks_playcount", metricName="rmse",labelCol="topTracks_playcount"):
    evaluations = []
    weights = [1.0] * k
    splits = data.randomSplit(weights)
    for i in range(0, k):  
        testingSet = splits[i]
        trainingSet = spark.createDataFrame(sc.emptyRDD(), data.schema)
        for j in range(0, k):
            if i == j:
                continue
            else:
                trainingSet = trainingSet.union(splits[j])
        als = ALS(userCol=userCol, itemCol=itemCol, ratingCol=ratingCol)
        model = als.fit(trainingSet)
        predictions = model.transform(testingSet)
        evaluator = RegressionEvaluator(metricName=metricName, labelCol=labelCol, predictionCol="prediction")
        evaluation = evaluator.evaluate(predictions.na.drop())
        print ("Loop " + str(i+1) + ": " + metricName + " = " + str(evaluation))
        evaluations.append(evaluation)
    return sum(evaluations)/float(len(evaluations))

print ("RMSE = " + str(kfoldALS(sub_rating_data, k=2)))

In [28]:
df = spark.createDataFrame([("A", 20), ("B", 30), ("D", 80)],["Letter", "Number"])
df.show()

+------+------+
|Letter|Number|
+------+------+
|     A|    20|
|     B|    30|
|     D|    80|
+------+------+



In [29]:
f = df.groupBy().sum().collect()
f

[Row(sum(Number)=130)]